In [7]:
from langchain_groq import ChatGroq

In [8]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [ ]:

from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-33460")
page_data = loader.load().pop().page_content
print(page_data)

In [11]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [12]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Job Poster',
 'experience': 5,
 'skills': ['Python', 'JavaScript', 'HTML/CSS'],
 'description': 'This is a job posting'}

In [13]:
type(json_res)

dict

In [14]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [16]:
import uuid
%pip install chromadb
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

  Obtaining dependency information for chromadb from https://files.pythonhosted.org/packages/80/4c/ee62b19a8daeed51e3c88c84b7da6047a74b786e598be3592b67a286d419/chromadb-0.5.5-py3-none-any.whl.metadata
  Obtaining dependency information for build>=1.0.3 from https://files.pythonhosted.org/packages/91/fd/e4bda6228637ecae5732162b5ac2a5a822e2ba8e546eb4997cde51b231a3/build-1.2.2-py3-none-any.whl.metadata
  Using cached build-1.2.2-py3-none-any.whl.metadata (6.2 kB)
  Obtaining dependency information for chroma-hnswlib==0.7.6 from https://files.pythonhosted.org/packages/0d/19/aa6f2139f1ff7ad23a690ebf2a511b2594ab359915d7979f76f3213e46c4/chroma_hnswlib-0.7.6-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for fastapi>=0.95.2 from https://files.pythonhosted.org/packages/d0/80/17037f322c280efbc623e341358d38d0299c6ee899d619a879b3593aa6da/fastapi-0.114.0-py3-none-any.whl.metadata
  Obtaining dependency information for uvicorn[standard]>=0.18.3 from https://files.pytho

/Users/amuzz/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [31:59<00:00, 43.3kiB/s]  


In [18]:
job = json_res
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/full-stack-js-portfolio'},
  {'links': 'https://example.com/typescript-frontend-portfolio'}],
 [{'links': 'https://example.com/wordpress-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}]]

In [19]:
job

{'role': 'Job Poster',
 'experience': 5,
 'skills': ['Python', 'JavaScript', 'HTML/CSS'],
 'description': 'This is a job posting'}

In [20]:
job = json_res
job['skills']

['Python', 'JavaScript', 'HTML/CSS']

In [21]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Solutions for Your Job Posting Needs

Dear Hiring Manager,

I came across your job posting for a Job Poster role, requiring expertise in Python, JavaScript, and HTML/CSS. I'd like to introduce you to AtliQ, an AI & Software Consulting company that can help you find the perfect candidate or even take care of the job requirements in-house.

At AtliQ, we have a proven track record of empowering enterprises with tailored solutions, fostering scalability, process optimization, cost reduction, and heightened overall efficiency. Our team of experts has extensive experience in developing and integrating automated tools to streamline business processes.

In terms of technical expertise, we have a strong portfolio in Python development, including machine learning and data science applications (https://example.com/ml-python-portfolio, https://example.com/python-portfolio). We also have expertise in full-stack JavaScript development, including frontend and backend solutions (https: